In [1]:
import pandas as pd
import numpy as np
import re
pd.options.display.max_columns = None

In [ ]:
df = pd.read_csv("productos.csv")

In [ ]:
# Extraer patrones de peso en Kg y gr en columnas temporales
df['peso_kg'] = df['nombre'].str.extract(r'(\d+[,.]?\d*\s?[Kk][gG])', expand=False)
df['peso_gr'] = df['nombre'].str.extract(r'(\d+\s?gr)', expand=False)

# combinar los valores existentes
df['peso'] = df['peso'].combine_first(df['peso_kg']).combine_first(df['peso_gr'])

# Eliminar las columnas temporales
df.drop(columns=['peso_kg', 'peso_gr'], inplace=True)

In [ ]:
df['peso'] = df['peso'].str.replace('.', ',')
# Eliminar espacios en blanco al principio y al final, y entre medio de cada valor en 'peso'
df['peso'] = df['peso'].str.strip().str.replace(r'\s+', '', regex=True)

In [ ]:
#creo una nueva columna, que me indicará si la arcilla es chamotada o no
df['chamota'] = df.apply(
    lambda row: (
        True if row['categoria'] == 'Arcillas' and re.search(r'\d+[-,.]?\d*mm|Chamot|chamot', row['nombre']) and 'sin chamota' not in row['nombre'].lower() 
        else False if row['categoria'] == 'Arcillas' and 'sin chamota' in row['nombre'].lower() 
        else None
    ), axis=1
)

In [ ]:
# Condiciones y opciones para la columna 'tipo'
condition_1 = df['categoria'] == 'Arcillas'
condition_2 = df['nombre'].str.contains('porcelana|Porcelana', case=False, na=False)
condition_3 = df['nombre'].str.contains('Refract|refract', case=False, na=False)
condition_4 = df['nombre'].str.contains('Gres|gres', case=False, na=False)
condition_5 = df['nombre'].str.contains('Terracota', case=False, na=False)

# Crear la columna 'tipo' basada en las condiciones
df['tipo'] = np.select(
    [condition_1 & condition_2, condition_1 & condition_3, condition_1 & condition_4, condition_1 & condition_5],
    ['Porcelana', 'Refractario', 'Gres', 'Terracota'],
    default=None
)

In [ ]:
# Expresión regular para encontrar el formato de capacidad en litros con "L" obligatoria
pattern = re.compile(r'(\d+(?:[.,]\d+)?)\s?[lL][iI]?[tT]?[rR]?[oO]?[sS]?|(\d+(?:[.,]\d+)?\s?[lL])')

# Función para extraer la capacidad y formatearla adecuadamente
def extract_capacidad(nombre):
    match = pattern.search(nombre)
    if match:
        number = re.findall(r'\d+(?:[.,]\d+)?', match.group(0))[0]
        return number.replace(',', '.') + 'L'  # Ensure consistent formatting
    return None

# Aplicar la función a la columna nombre y crear la nueva columna capacidad
df['capacidad'] = df['nombre'].apply(extract_capacidad)

In [ ]:
# Expresión regular para encontrar los formatos de tamaño
pattern_size = re.compile(r'(\d+(?:[.,]?\d*)\s?[xX×]\s?\d+(?:[.,]?\d*)(?:\s?[xX×]\s?\d+(?:[.,]?\d*)?)?\s?(?:mm|cm|m)?)|(\d+(?:[.,]?\d*)\s?(?:mm|cm|m))')

# Función para extraer el tamaño
def extract_size(nombre):
    match = pattern_size.search(nombre)
    if match:
        return match.group(0).replace('×', 'x').replace(' ', '')
    return None

# Aplicar la función a la columna nombre y crear la nueva columna tamaño
df['tamaño'] = df['nombre'].apply(extract_size)

In [ ]:
# Función para actualizar la subcategoría
def actualizar_subcategoria(row):
    if row['categoria'] == 'Maquinaria':
        if 'Galletera' in row['nombre']:
            return 'Galleteras'
        elif 'Extrusionadora' in row['nombre']:
            return 'Extrusionadoras'
        elif 'Torno' in row['nombre'] or 'torno' in row['nombre']:
            return 'Tornos'
        elif 'Laminadora' in row['nombre']:
            return 'Laminadoras'
    return row['subcategoria']

# Aplicar la función al DataFrame
df['subcategoria'] = df.apply(actualizar_subcategoria, axis=1)

In [ ]:
# Función para actualizar la subcategoría
def actualizar_subcategoria(row):
    if row['categoria'] == 'Herramientas':
        if 'Vaciador' in row['nombre']:
            return 'Vaciadores'
        elif "VACIADOR" in row['nombre']:
            return 'Vaciadores'
        elif 'Tamiz' in row['nombre']:
            return 'Tamices'
        elif "Pera" in row['nombre']:
            return "Peras"
        elif "Palillo" in row['nombre']:
            return "Palillos modelar"
        elif "Perforador" in row['nombre']:
            return "Perforadores"
    return row['subcategoria']

# Aplicar la función al DataFrame
df['subcategoria'] = df.apply(actualizar_subcategoria, axis=1)

In [ ]:
# Aplicar el cambio de subcategoría según la condición
filtro = (df['categoria'] == 'Lápices y tizas cerámicas') & (df['subcategoria'] == 'Elementos protección')
df.loc[filtro, 'subcategoria'] = 'Tizas'

In [ ]:
#para terminar paso los datos de estas columnas a categóricas
df['categoria'] = df['categoria'].astype('category')
df['subcategoria'] = df['subcategoria'].astype('category')
df['tienda'] = df['tienda'].astype('category')
df['tipo'] = df['tipo'].astype('category')
df['tienda'] = df['tienda'].astype('category')
df['chamota'] = df['chamota'].astype('boolean')

In [ ]:
df.to_csv("productos.csv", index=False)